In [1]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets

import pandas as pd
import numpy as np
from scipy import special

py.offline.init_notebook_mode(connected=True)

In [2]:
partial_pdf_file = './UCl3_rmc_try1_3_9/UCl3_900_neutron1_PDFpartials.csv'

In [3]:
pp_file = open(partial_pdf_file, "r")
lines = pp_file.readlines()
pp_file.close()

partials = [item.strip() for item in lines[0].split(",")[1:]]
partial_pdfs = dict()
partial_pdfs["r"] = []
for i in range(len(partials)):
    partial_pdfs[partials[i]] = []
for line in lines[1:]:
    if line:
        partial_pdfs["r"].append(float(line.split(",")[0]))
        for i in range(len(partials)):
            partial_pdfs[partials[i]].append(float(line.split(",")[i + 1]))

In [5]:
layout = go.Layout(
    title='Partial PDFs Inspection',
    yaxis=dict(
        title='g(r)',
        autorange=False
    ),
    xaxis=dict(
        title='r (angstrom)',
        autorange=False
    ),
    width=1000,
    height=700,
    autosize=False
)


def boltzmann(x, A1, A2, r0, delta_r):
    y = list()
    for item in x:
        tmp = (item - r0) / delta_r
        tmp = 700 if tmp > 700 else tmp
        y.append((A1 - A2) / (1. + np.exp(tmp)) + A2)
    
    return y


def update_plot(signal, A1, A2, r0, delta_r, xmin, xmax, ymin, ymax, BZ_m, X_m, Y_m):
    
    trace_data = go.Scatter(
        x=partial_pdfs["r"],
        y=partial_pdfs[signal],
        mode="lines",
        name=f"{signal} - Data",
        line=dict(
            shape='spline'
        )
    )

    lb1.value = "Manual Input Boltzmann Parameters: " + BZ_m
    lb2.value = "Manual Input X-range: " + X_m
    lb3.value = "Manual Input Y-range: " + Y_m
    
    if len(BZ_m.split()) >= 4:
        valid_input = True
        for item in BZ_m.split():
            if not item.strip().replace('.', '', 1).replace('-', '', 1).replace(' ', '').isdigit():
                valid_input = False
                break
        if valid_input:
            A1 = float(BZ_m.split()[0])
            A2 = float(BZ_m.split()[1])
            r0 = float(BZ_m.split()[2])
            delta_r = float(BZ_m.split()[3])
            if abs(delta_r) < 1E-2:
                delta_r = 0.01
    
    trace_tail = go.Scatter(
        x=partial_pdfs["r"],
        y=boltzmann(partial_pdfs["r"], A1, A2, r0, delta_r),
        mode="lines",
        name=f"{signal} - Tail",
        line=dict(
            shape='spline'
        )
    )
    
    data = [trace_data, trace_tail]
    fig = go.Figure(data=data, layout=layout)
    
    if len(X_m.split(",")) == 2:
        if X_m.split(",")[0].strip().replace('.', '', 1).replace('-', '', 1).replace(' ', '').isdigit():
            xmin = float(X_m.split(",")[0])
        if X_m.split(",")[1].strip().replace('.', '', 1).replace('-', '', 1).replace(' ', '').isdigit():
            xmax = float(X_m.split(",")[1])
    if len(Y_m.split(",")) == 2:
        if Y_m.split(",")[0].strip().replace('.', '', 1).replace('-', '', 1).replace(' ', '').isdigit():
            ymin = float(Y_m.split(",")[0])
        if Y_m.split(",")[1].strip().replace('.', '', 1).replace('-', '', 1).replace(' ', '').isdigit():
            ymax = float(Y_m.split(",")[1])
            
    if xmax < xmin:
        tmp = xmin
        xmin = xmax
        xmax = tmp
    if ymax < ymin:
        tmp = ymin
        ymin = ymax
        ymax = tmp
        
    fig.update_xaxes(range=[xmin, xmax])
    fig.update_yaxes(range=[ymin, ymax])
    
    py.offline.iplot(fig)
    
    
style = {'description_width': 'initial'}
# Tail function parameters
signal = widgets.Dropdown(options=partials, value=partials[0], description='Partials')
A1 = widgets.FloatSlider(min=-1, max=5, value=0, step=0.001, description='A1',
                         layout=widgets.Layout(width='30%'), style=style,
                         readout_format='.3f')
A2 = widgets.FloatSlider(min=0, max=10, value=1, step=0.001, description='A2',
                         layout=widgets.Layout(width='30%'), style=style,
                         readout_format='.3f')
r0 = widgets.FloatSlider(min=0, max=5, value=2.0, step=0.001, description='r0',
                         layout=widgets.Layout(width='30%'), style=style,
                         readout_format='.3f')
delta_r = widgets.FloatSlider(min=0.01, max=0.5, value=0.001, step=0.01, description='delta_r',
                              layout=widgets.Layout(width='30%'), style=style,
                              readout_format='.3f')

# Plot range parameters
xmin = widgets.FloatSlider(min=0, max=50, value=0, step=0.01, description='X-range slider-1',
                           layout=widgets.Layout(width='30%'), style=style, readout_format='.3f')
xmax = widgets.FloatSlider(min=0, max=50, value=50, step=0.01, description='X-range slider-2',
                           layout=widgets.Layout(width='30%'), style=style, readout_format='.3f')
ymin = widgets.FloatSlider(min=-5, max=50, value=0, step=0.01, description='Y-range slider-1',
                           layout=widgets.Layout(width='30%'), style=style, readout_format='.3f')
ymax = widgets.FloatSlider(min=-5, max=50, value=20, step=0.01, description='Y-range slider-2',
                           layout=widgets.Layout(width='30%'), style=style, readout_format='.3f')

# Manual inputs
BZ_m = widgets.Text(value='',
                    placeholder='0 1 2.0 0.1',
                    description='BZ Params: ',
                    disabled=False)
X_m = widgets.Text(value='',
                   placeholder='0, 50',
                   description='X-range: ',
                   disabled=False)
Y_m = widgets.Text(value='',
                   placeholder='0, 20',
                   description='Y-range: ',
                   disabled=False)

lb1 = widgets.Label()
display(lb1)
lb2 = widgets.Label()
display(lb2)
lb3 = widgets.Label()
display(lb3)

widgets.interactive(update_plot, signal=signal, A1=A1, A2=A2, r0=r0, delta_r=delta_r,
                    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, BZ_m=BZ_m, X_m=X_m, Y_m=Y_m)

Label(value='')

Label(value='')

Label(value='')

interactive(children=(Dropdown(description='Partials', options=('U-U', 'U-Cl', 'Cl-Cl'), value='U-U'), FloatSl…